📖 출처: 텐서플로 2와 머신러닝으로 시작하는 자연어 처리

# 텍스트 유사도(Text Similarity)
    - 문장이 유사한지 측정하는 방법
    - 유사도를 판단하는 척도는 매우 주관적이기 때문에 데이터를 구성하기가 쉽지 않고 정량화하는 데 한계가 있다  
    => 최대한 정량화해서 모델을 만드는 것이 중요

## 유사도를 측정하기 위해 정량화하는 방법
    - 단순히 같은 단어의 개수를 사용해서 유사도를 판단하는 방법
    - 형태소로 나누어 형태소를 비교하는 방법
    - 자소 단위로 나눈어 단어를 비교하는 방법 등등등

**단어, 형태소, 유사도의 종류에 상관 없이, 텍스트를 벡터화한 후 벡터화된 각 문장간의 유사도를 측정하는 방법에 대해 공부함**  
**주로 쓰이는 유사도 측정 방법 : 자카드 유사도, 유클리디언 유사도, 맨하탄 유사도, 코사인 유사도**

----

예시문장
- 휴일인 오늘도 서쪽을 중심으로 폭염이 이어졌는데요, 내일은 반가운 비 소식이 있습니다.
- 폭염을 피해서 휴일에 놀러왔다가 갑작스런 비로 인해 망연자실하고 있습니다.

## 단어 벡터화(TF-IDF : Term Frequency - Inverse Document Frequency)

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentence = ("휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.", "폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니다.")

# 객체 생성
tfidf_vectorizer = TfidfVectorizer()

# 문장 벡터화 진행
tfidf_matrix = tfidf_vectorizer.fit_transform(sentence)

# 각 단어
text = tfidf_vectorizer.get_feature_names()

# 각 단어의 벡터 값
idf = tfidf_vectorizer.idf_
print(dict(zip(text, idf)))

{'갑작스런': 1.4054651081081644, '내일': 1.4054651081081644, '놀러왔다가': 1.4054651081081644, '망연자실': 1.4054651081081644, '반가운': 1.4054651081081644, '서쪽': 1.4054651081081644, '소식': 1.4054651081081644, '오늘': 1.4054651081081644, '으로': 1.4054651081081644, '이어졌는데요': 1.4054651081081644, '인해': 1.4054651081081644, '있습니다': 1.0, '중심': 1.4054651081081644, '폭염': 1.0, '피해서': 1.4054651081081644, '하고': 1.4054651081081644, '휴일': 1.0}


=> TF-IDF로 벡터화한 값은 자카드 유사도를 제외한 유사도 측정에 모두 사용할 것임.

- [비가 나오지 않는 현상](https://github.com/NLP-kr/tensorflow-ml-nlp/issues/13)  
sklearn 에 TF-IDF API에서는 한글 한음절 (예: "비", "눈" 등) 과 알파벳 한개(예: "a", "t"등) 은 분석 내용에서 포함되지 않아 위와 같은 결과가 발생하고 있으며, API 특성으로 보임

## 유사도 측정 방법

### 자카드 유사도(Jaccard Similarity)
    - 두 문장을 각각 단어의 집합으로 만든 뒤 두 집합을 통해 유사도를 측정하는 방식 중 하나이다.
    - 유사도를 측정하는 방법은 두 집합의 교집합인 공통된 단어의 개수를 두 집합의 합집합 즉, 전체 단어의 개수로 나눈다.
    - 결과값은 공통의 원소의 개수에 따라 0고 1사이의 값이 나올 것이고, 1에 가까울수록 유사도가 높다.

### 코사인 유사도(Cosine Similarity)
    - 두 개의 벡터값에서 코사인 각도를 구하는 방법이며, 일반적으로 성능이 좋기 때문에 가장 널리 쓰이는 방법 중 하나이다.
    - 코사인 유사도 값은 -1과 1사이의 값을 가지고 1에 가까울수록 유사하다는 것을 의미한다.
    - 단순히 좌표 상의 거리를 구하는 다른 유사도 측정과 달리 두 벡터의 각도를 구하므로 방향성의 개념이 더해진다.
    - 두 문장이 유사하다면 같은 방향을 가리키고, 그렇지 않다면 직교로 표현 될 것이다.

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째와 두 번째 문장 비교
print(cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2]))

[[0.17952266]]


### 유클리디언 유사도(Euclidean Distance) or L2 거리(L2-Distance)
    - 가장 기본적인 거리를 측정하는 유사도 공식
    - 유클리디언 거리 = L2 거리 : N차원 공간에서 두 점 사이의 최단 거리를 구하는 접근법
    - 유클리디언 유사도는 단순히 두 점 사이의 거리를 뜻하므로, 1 이상의 값을 가지지 않을 수도 있다.

In [3]:
from sklearn.metrics.pairwise import euclidean_distances

print(euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2]))

[[1.28099753]]


    - 앞서 구한 벡터를 일반화한 후 다시 유클리디언 유사도를 측정하면 0과 1 사이의 값을 가진다.
    - L1 정규화 방법 : 각 벡터안의 요소값을 모두 더한 것의 크기가 1이 되도록 벡터의 크기를 조절하는 방법
    즉, 벡터의 모든 값을 더한 뒤, 이 값으로 각 벡터의 값을 나눈다.
    - 유클리디언 유사도를 측정할 때는, 편의를 위해 정규화한 후 측정하는 방법도 있다는 점을 기억하자!

In [4]:
import numpy as np

def l1_normalize(v):
    norm = np.sum(v)
    return v / norm

tfidf_norm_l1 = l1_normalize(tfidf_matrix)

print(euclidean_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2]))

[[0.20491229]]


### 맨하탄 유사도(Manhattan Similarity) or L1 거리(L1-Distance)
    - 사각형 격자로 이뤄진 지도에서 출발점에서 도착점까지를 가로지르지 않고 갈 수 있는 최단거리를 구하는 공식
    - 맨하탄 거리 = L1 거리
    - 맨하탄 유사도 또한 유클리디언 유사도와 마찬가지로 값이 계속 커질 수 있다.
    - 따라서 0과 1사이의 값을 갖도록 L1 정규화 방법을 사용한 벡터 값으로 유사도를 측정한다.

In [5]:
from sklearn.metrics.pairwise import manhattan_distances

print(manhattan_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2]))

[[0.77865927]]


---

## 결론

    - 맨하탄 유사도로 측정했을 때 유사도가 가장 높게 나왔다.
    - 측정 방법에 따라 크게 유사도가 달라질 수 있으므로 의도하고자 하는 방향에 맞는 유사도 측정 방법을 고르는 것이 매우 중요하다.